Test and debug functions from spikingDataUtilities

In [ ]:
import numpy as np
import spikingDataUtilities as su
%load_ext autoreload
%autoreload 2

In [ ]:
# test restrict

# first column of samples is assumed to be time stamps at which all other columns are evaluated
samples = np.array([[1,5],[3,8],[4,-1.5],[7,2]])
# every row of intervals is an interval, sample time stamps must fall in at least one interval
intervals = np.array([[1,3.2],[6,8]])
su.restrict(samples,intervals)

array([[ 1. ,  5. ],
       [ 3. ,  8. ],
       [ 4. , -1.5],
       [ 7. ,  2. ]])